In [1]:
import torch
import torchvision
import torchvision.transforms as transforms


In [2]:

from pamap2_load import get_pamap2
import numpy as np

#from torch.autograd import Variable as V

import plotly.express as px

from PIL import Image

import matplotlib
import matplotlib.pyplot as plt


import numpy as np

import ripser
import scipy.io as sio
import os
from scipy import sparse
from persim import PersImage
import sys
from ripser import ripser


# Import Scikit-Learn tools
from numpy import inf

In [3]:
NUM_WORKERS = 2

SEED = 1


In [7]:

def get_PAMAP2_TDA_data3(test_id=0, dataset_dir='data/', batch_size=64):

    np.random.seed(222)

    x_train, y_train, x_test, y_test = get_pamap2(test_id, dataset_dir)

    x_train = x_train.reshape((x_train.shape[0],x_train.shape[2],x_train.shape[3]))

    print("x_train",x_train.shape)

    x_test = x_test.reshape((x_test.shape[0],x_test.shape[2],x_test.shape[3])) #numdata, window, ch

    print("x_test",x_test.shape)
    
    
    threshold = 0.01
    pixel = 64
    spreadval = 0.015
    rangemin = -1
    rangemax = 1
    pim = PersImage(pixels=[pixel,pixel], spread=spreadval, specs={"minBD": rangemin, "maxBD": rangemax})
        
    
    ####### define test data #######
    result_all_data2 = []    
    result_all_ydata2 = []  
    
    for idx, xdata in enumerate(x_test[:]):
        
        result_data2 = []
        y = y_test[idx]
        for ch in range(x_test.shape[2]):

            x = xdata[:,ch]
            x = x.reshape((-1))

            #####make persistence diagram
    
            # Extracting Needed Info
            N = len(x)
            t = np.arange(N)
    
            #Sublevelset Filtration
            # Add edges between adjacent points in the time series, with the "distance"
            # along the edge equal to the max value of the points it connects
            I = np.arange(N-1)
            J = np.arange(1, N)
            V = np.maximum(x[0:-1], x[1::])
    
            # Add vertex birth times along the diagonal of the distance matrix
            I = np.concatenate((I, np.arange(N)))
            J = np.concatenate((J, np.arange(N)))
            V = np.concatenate((V, x))
    
            #Create the sparse distance matrix
            D = sparse.coo_matrix((V, (I, J)), shape=(N, N)).tocsr()
            dgm0 = ripser(D, maxdim=0, distance_matrix=True)['dgms'][0] ##Sparse Filtration
                        
            #plt.figure(figsize=(12, 4))
            #plot_diagrams(dgm0, show=False)
            #plt.title("Sparse Filtration")
            #plt.tight_layout()
            #plt.show() 
            
                
            dgm0 = dgm0[dgm0[:, 1]-dgm0[:, 0] > 1e-3, :]
            allgrid = np.unique(dgm0.flatten())
            allgrid = allgrid[allgrid < np.inf]
            xs = np.unique(dgm0[:, 0])
            ys = np.unique(dgm0[:, 1])
            ys = ys[ys < np.inf]
    
            # Removing Infinity Points
            where_are_inf = np.isinf(dgm0)
            dgm0 = dgm0[~where_are_inf[:,1]]
    
            # Computing Lifetimes for Thresholding of Persistence Diagrams
            lifetime = np.ones(len(dgm0))
    
            for h in range(len(dgm0)):
                lifetime[h] = dgm0[h][1] - dgm0[h][0]
        
            #Keeps only points greater than reduction value
            isvalid = np.greater(lifetime, threshold) 
            reduced_pd = dgm0[isvalid] #code info
            
            imgs = pim.transform(reduced_pd)
    
            if np.max(imgs) > 0:
                imgs = imgs/np.max(imgs)
    
            result_data2.append(imgs)
        #print("result_data",np.asarray(result_data))
        
        arr_data2 = np.asarray(result_data2)
        #print("tep",tep.shape, tep.max())
        result_all_data2.append(arr_data2)
        result_all_ydata2.append(y)
        
        
        if idx % 1000 == 0:
            print("Test idx == ", idx)
            
    arr_all_data2 = np.asarray(result_all_data2)
    arr_all_ydata2 = np.asarray(result_all_ydata2)    
    
    np.save('./Result_img_id/'+'xtest_org'+str(test_id),arr_all_data2)
    np.save('./Result_img_id/'+'ytest_org'+str(test_id),arr_all_ydata2)
         
    
    return "end"


In [8]:
if __name__ == "__main__":


    print("get_PAMAP2_TDA_data")
    
    for i in range(0,9):
        running = get_PAMAP2_TDA_data3(i) #get_PAMAP2_TDA_data3a
                        
    print("------"*20)
    print("--PI generation finished--")


get_PAMAP2_TDA_data
train_id : [1, 2, 3, 4, 5, 6, 7, 8] test_id : 0
-----------------train subject data---------------------
1 87783
2 58113
3 77141
4 90814
5 83366
6 77592
7 87368
8 2131
-----------------test subject data---------------------
0 83319
25223 3726 562177 83319
train: 25223   test: 3726   class: 12
-------------Train data #-----------------
0 - 530
1 - 2470
2 - 2418
3 - 2517
4 - 3248
5 - 1143
6 - 2110
7 - 2515
8 - 1468
9 - 1296
10 - 2279
11 - 3229
-------------Test data #-----------------
0 - 191
1 - 407
2 - 352
3 - 325
4 - 333
5 - 318
6 - 352
7 - 302
8 - 233
9 - 217
10 - 343
11 - 353
x_train (25223, 100, 40)
x_test (3726, 100, 40)
PersImage(pixels=[64, 64], spread=0.015, specs={'minBD': -1, 'maxBD': 1}, kernel_type="gaussian", weighting_type="linear")
Test idx ==========  0


/tmp/ipykernel_17866/463931237.py:21: DeprecationWarning: Call to deprecated class PersImage. (Replaced with the class `persim.PersistenceImager`.) -- Deprecated since version 0.1.5.
  pim = PersImage(pixels=[pixel,pixel], spread=spreadval, specs={"minBD": rangemin, "maxBD": rangemax})


train_id : [0, 2, 3, 4, 5, 6, 7, 8] test_id : 1
-----------------train subject data---------------------
0 83319
2 58113
3 77141
4 90814
5 83366
6 77592
7 87368
8 2131
-----------------test subject data---------------------
1 87783
25024 3925 557713 87783
train: 25024   test: 3925   class: 12
-------------Train data #-----------------
0 - 525
1 - 2527
2 - 2435
3 - 2459
4 - 3093
5 - 1326
6 - 2086
7 - 2371
8 - 1448
9 - 1292
10 - 2313
11 - 3149
-------------Test data #-----------------
0 - 196
1 - 350
2 - 335
3 - 383
4 - 488
5 - 135
6 - 376
7 - 446
8 - 253
9 - 221
10 - 309
11 - 433
x_train (25024, 100, 40)
x_test (3925, 100, 40)
PersImage(pixels=[64, 64], spread=0.015, specs={'minBD': -1, 'maxBD': 1}, kernel_type="gaussian", weighting_type="linear")
Test idx ==========  0
train_id : [0, 1, 3, 4, 5, 6, 7, 8] test_id : 2
-----------------train subject data---------------------
0 83319
1 87783
3 77141
4 90814
5 83366
6 77592
7 87368
8 2131
-----------------test subject data------------------